# Lab 02: Introduction to Text Preprocessing & the Spacy Toolkit

### Objectives:
1. Get familiar with basic text preprocessing pipelines
2. Get familiar with regular expressions, and the `re` package in Python
3. Evaluate the lexical diversity of the data in each category within the 20 News Groups Dataset
4. Use normalized BOW features to evaluate text similarity using the KL-divergence

### Required Reading:

1. https://universaldependencies.org/u/pos/
2. https://spacy.io/api/annotation#pos-tagging
3. https://spacy.io/api/annotation#dependency-parsing

# Part I: Introduction to Spacy

### Download Spacy's base English language *pipeline* components

``$ python -m spacy download en_core_web_sm``

What is a Spacy *pipeline*? A Spacy pipeline is an extensible tool that streamlines many of the common tasks in NLP, such as tokenization, part-of-speech tagging, named entity recognition, stemming, lemmatizing, and parsing. It also has custom pipeline components specifically for transformers. It is built for production use; much thought and care has gone into its API and implementation. You can actually configure Spacy to use some of the statistical models that we will discover in this class; for now we're just going to cover some of the basics.

In [2]:
import spacy

pipeline = spacy.load('en_core_web_sm')

### Download the 20 News Groups dataset using the sklearn package

This data consists of news articles from 20 different categories. 

In [3]:
from sklearn.datasets import fetch_20newsgroups
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

ng_train = fetch_20newsgroups(subset='train')
ng_test = fetch_20newsgroups(subset='test')
ng_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Get the number of training & test examples

In [4]:
len(ng_train.data), len(ng_test.data)

(11314, 7532)

### Take a peek at the first document and its label

In [7]:
ng_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [8]:
label_idx = ng_train.target[0]
ng_train.target_names[label_idx]

'rec.autos'

### Evaluate Spacy's recognition of entities, POS

In [9]:
from pprint import pprint

doc = pipeline(ng_train.data[0])
for i, token in enumerate(doc):
    pprint({"text": token.text,
            "lemma": token.lemma_,
            "POS": token.pos_,
            "tag": token.tag_,
            "dep": token.dep_,
            "shape": token.shape_,
            "is_alpha": token.is_alpha,
            "is_stop": token.is_stop})
    if i == 3:
        break

{'POS': 'ADP',
 'dep': 'prep',
 'is_alpha': True,
 'is_stop': True,
 'lemma': 'from',
 'shape': 'Xxxx',
 'tag': 'IN',
 'text': 'From'}
{'POS': 'PUNCT',
 'dep': 'punct',
 'is_alpha': False,
 'is_stop': False,
 'lemma': ':',
 'shape': ':',
 'tag': ':',
 'text': ':'}
{'POS': 'PROPN',
 'dep': 'pobj',
 'is_alpha': False,
 'is_stop': False,
 'lemma': 'lerxst@wam.umd.edu',
 'shape': 'xxxx@xxx.xxx.xxx',
 'tag': 'NNP',
 'text': 'lerxst@wam.umd.edu'}
{'POS': 'PUNCT',
 'dep': 'punct',
 'is_alpha': False,
 'is_stop': False,
 'lemma': '(',
 'shape': '(',
 'tag': '-LRB-',
 'text': '('}


### Visualize Spacy's dependency parse

In [10]:
from spacy import displacy
displacy.render(doc, style='dep')

### Let's define a preprocessing function that cleans our data

You'll notice that even the lemmatized text contains meaningless tokens. In the real world you're never going to get around having to do some feature engineering. In NLP this often means writing some regexes to transform text into a usable format. This has become less important in the deep learning era, but applying domain specific knowledge is always beneficial. In the case of this dataset, we have text that originated in news feeds, some of which is messy. There are email and url addresses, grammatical errors, and a lot puntuation and uninformative characters (e.g., the newline character `\n`). Below is a function that does some very basic regex (regular expression) matching to strip out emails, urls, punctuation, and other junk.

In [11]:
import re
from spacy.language import Language


# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "numbr"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "),   # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

train_text = ng_train.data
test_text = ng_test.data
for repl in replace:
    train_text = [re.sub(repl[0], repl[1], text) for text in train_text]
    test_text = [re.sub(repl[0], repl[1], text) for text in test_text]

@Language.component("ng20")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)

pipeline.add_pipe("ng20");

#### Peek at our processing pipeline

In [12]:
pipeline.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'ng20': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
 

### Now pass each training and test document through the pipeline

In [13]:
docs_train = [pipeline(doc) for doc in train_text[:500]]
docs_test = [pipeline(doc) for doc in test_text[:500]]

### Let's look at that first document following this transformation and compare it to the original text

In [71]:
ng_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [73]:
docs_train[0]

'email thing subject car nntp posting host racnumbr wam umd edu organization university maryland college park lines numbr wonder enlighten car see day numbr door sport car look late numbrs early numbrs call bricklin door small addition bumper separate rest body know tellme model engine spec year production car history info funky look car e mail thanks il bring neighborhood lerxst'

# Part II: Lexical diversity

Sometimes it's useful to understand how diverse is the language in some body of text. Once simple heuristic to evaluate diversity is as follows: 

$$ lexical\_diversity = \frac{ len(set(all\_words\_in\_doc)) }{ len(doc) }$$

Find the set of all words observed in the document, and divide it by the number of total words in the document. Let's use this to evalute the diversity of each category in the 20NG dataset.

### (5 pts) Task I: 
In the cell below, compute the diversity of each category in the 20NG dataset using the above heuristic

In [81]:
# Your code goes here
import numpy as np
content_list=[]
for index in range(0,20):
    index_list= np.where(ng_train.target[:500]==index)
    index_list=list(index_list[0])
    content=""
    for item in index_list:      
        content = content+" "+docs_train[item]
    content_list.append(content.split())

In [84]:
content_set=[]
for item in content_list:
    content_set.append(set(item))

In [87]:
lexical_diversity=[]
for index in range(len(content_list)):
    lexical_diversity.append(len(content_set[index])/len(content_list[index]))
lexical_diversity

[0.3058645096056623,
 0.37962659679004257,
 0.4647410037114733,
 0.4031083050024284,
 0.34853603603603606,
 0.3611748508490133,
 0.4002859185132237,
 0.2895543503890592,
 0.410958904109589,
 0.28291896869244937,
 0.34358091490408266,
 0.412396990326048,
 0.3763945977686436,
 0.3809985458070771,
 0.3129588254069582,
 0.3238221632382216,
 0.37437701553796543,
 0.33233900684330586,
 0.46843725943033104,
 0.44962302947224125]

### Explain these scores: 

1. Is this result real or an artifact of some underlying problem with our data? 
2. What might you do to better evaluate lexical diversity on this data using this scoring function?
3. Is this heuristic a good metric for lexical diversity in general?

answer: I think the results is an artifact of some underlying problem with our data. From some examples of processing results, there are still many meaningless words or repeated words in the processed data, which can not accurately calculate the statistical information of each category.If we want to measure vocabulary diversity more accurately, we need to further feature the data and delete some meaningless words and symbols

### Entropy 
Entropy is another, perhaps more principled, way by which we can evaluate how diverse, or varied, is a piece of text. Recall the definition of Entropy, $H(P(x))$:

$$ H(P(x)) = \sum_{i=1}^{N} -P(x_{i}) \log P(x_{i}) $$

In the Bag-of-Words (BOW) feature representation of a document, each document is represented by a word count vector, ${x}_{i} \in \mathbb{R}^{N}$ where $N$ is the cardinality of the set of words in the document.

### (5 pts) Task II:
In order to compute an entropy from this representation, you'll first need to convert those count vectors into probability distributions. Then compute the entropy of the word distributions for each news category.

In [93]:
# Your code goes here
import collections
import math
def get_counts(word_list):
    return collections.Counter(word_list)

def count_smoothing(freq_dist, vocabulary, alpha=1):
    return {w: freq_dist.get(w, 0) + alpha for w in vocabulary}

def create_prob_dist(count_dict):
    total_ct = sum(count_dict.values())
    p = {x: ct / total_ct for x, ct in count_dict.items()}
    return p

def entropy(p):
    h = 0
    # Calculate entropy value in nats for probability distribution `p`
    for x in p:
        h -= p[x] * math.log(p[x])
    return h

vocab=set()
for index in range(len(content_set)):
    vocab = vocab | content_set[index]

H_list=[]
pro_list=[]
for index in range(len(content_list)):
    ct = get_counts(content_list[index])
    ct_sm = count_smoothing(ct, vocab)
    pro = create_prob_dist(ct_sm)
    h= entropy(pro)
    pro_list.append(pro)
    H_list.append(h)

H_list


[9.486155857157394,
 9.666602047994308,
 8.912296592487694,
 9.731327601801164,
 9.613857696114788,
 9.70720619468539,
 9.653884466981365,
 9.408355916373065,
 9.644297600205283,
 9.316565214517405,
 9.553316908283552,
 9.714322336249687,
 9.634426981015402,
 9.608340339750345,
 9.461833365422937,
 9.50891989569167,
 9.643566597535843,
 9.50981203781284,
 9.732768726817916,
 9.775751478767736]

### Explain this result

1. What does it mean for a distribution to have high or low entropy?
2. Do these scores make intuitive sense? Any more or less so than the heuristic from Task I?
2. Is entropy a good metric for evaluating lexical diversity in general?

The greater the entropy, the more information, the richer the content, and the more vocabulary. the scores can make intuitive sense, I think it can make more intuitive sense than task1.If the data preprocessing is done well,the entropy is a good metric for evaluating lexical diversity in general

# Part III: Document Similarity

Throughout this course we will discuss the notion of *similarity* between texts and explore ways to measure it. This is a critical component of search and recommender systems. One such approach involves measuring how *close* two word distributions are using the notion divergence, which we discussed in the first lecture.

### (10 pts) Task III

Using the definition below, compute the KL-divergence, $K_{DL}$, between the word distributions in each category. This will result in a $K \times K$ matrix of divergence values.

$$ D_{KL}(P||Q) = \sum_{i=1}^{N} P(x_{i}) \log \frac{P(x_{i})}{Q(x_{i})} $$

In [94]:
# Your code goes here
def cross_entropy(p1, p2):
    """Calculate cross-entropy H(p1, p2) for two probability distributions
    represented as a mapping (dictionary) from word tokens toprobabilities
   """
    xh = 0
    # TODO -- Calculate cross-entropy value H(p1, p2) in nats
    for x in p1:
        xh -= p1[x] * math.log(p2[x])
    return xh


def kl_divergence(p1, p2):
    """Calculate Kullback-Leibler divergence D_{KL}(p1||p2) for two
    probability distributions represented as a mapping (dictionary)
    from word tokens to probabilities
    """
    kl = 0
    # TODO -- Calculate KL divergence D_{KL}(p1||p2) in nats
    kl = cross_entropy(p1, p2) - entropy(p1)
    return kl

K_matrix=[]
for i in range(len(pro_list)):                            
    K_matrix.append([])                               
    for j in range(len(pro_list)):                        
        K_matrix[i].append(0)

for i in range(len(pro_list)):   
    for j in range(len(pro_list)): 
        K_matrix[i][j] = kl_divergence(pro_list[i], pro_list[j])

K_matrix

[[0.0,
  0.22347644824987434,
  0.4643617582728652,
  0.24547271987178654,
  0.23802318614192153,
  0.24752216696491658,
  0.2663034748188515,
  0.2576341269147129,
  0.21340481074897255,
  0.2718623951837156,
  0.23406611679558686,
  0.2312066393111767,
  0.22684845289960975,
  0.2188550058183658,
  0.25474077681715457,
  0.1856397018975784,
  0.18593958860962623,
  0.22332775748823863,
  0.20799207310506596,
  0.1967204827175042],
 [0.18200595408479003,
  0.0,
  0.2885687753079207,
  0.09079663292154372,
  0.0938326764674251,
  0.08142048107707467,
  0.09852605848986329,
  0.12778065832483243,
  0.10128179757921707,
  0.15661779286798527,
  0.12548930380214607,
  0.12929877464414652,
  0.09094394328420563,
  0.1092037209703065,
  0.1447225151829663,
  0.15346147771959195,
  0.13510055602749915,
  0.1848403966907597,
  0.12175759856948076,
  0.0960879010273139],
 [0.6103865079930841,
  0.4422362508128135,
  0.0,
  0.542287939410425,
  0.46837594080446365,
  0.485985500635552,
  0.5120

### Explain this result

1. How did you handle any differences in the support for P and Q? What about when Q(x) = 0?
2. What does it mean for two distributions to have high or low divergence?
3. Do these similarity scores make sense intuitively?
4. Is the resultant $K \times K$ matrix symmetric? Why is this the case?
5. Is $D_{KL}$ a good measure of the similarity in this context?   

Generally speaking, the lower the divergence, the closer the two are.
Divergence can intuitively show the difference between two documents.
The divergence matrix is asymmetric, mainly becase:
KL (P||Q) is very large, which means that when the probability of P event is high, Q event does not necessarily have a high probability;
KL (Q||P) is very small, which means that when Q event has a high probability, P event also has a high probability.
One possible cause of this phenomenon is that Q is one of the many causes of P, so when Q occurs (high probability), P also occurs (high probability); P has other causes, so when p occurs, Q does not necessarily occur. Of course, understanding the dependency or inclusion relationship is another feasible idea: q is a subset of P, so p must occur when Q occurs, and when p occurs, some events in P\Q may occur, so Q does not necessarily occur.
I think Divergence is a good measure.